In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

print("--- ΕΚΚΙΝΗΣΗ ΠΛΗΡΟΥΣ ΑΝΑΛΥΣΗΣ (NUMERICAL + TEXTUAL) ---")

# 1. Ρυθμίσεις
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 7)
save_dir = os.getcwd() # Αποθήκευση στον τρέχοντα φάκελο

# 2. Φόρτωση Αρχείου (Διόρθωση για ; και encoding)
print("Φόρτωση δεδομένων...")
try:
    df = pd.read_csv('movies.csv', sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)
except:
    df = pd.read_csv('movies.csv', sep=';', encoding_errors='ignore', on_bad_lines='skip', low_memory=False)

# 3. Μετατροπή Αριθμητικών Στηλών (για τα Δέντρα)
cols_num = ['budget', 'popularity', 'runtime', 'revenue']
for col in cols_num:
    if col in df.columns:
        # Αντικαθιστούμε το κόμμα (,) με τελεία (.) αν υπάρχει σε αριθμούς και μετατρέπουμε
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# --- ΓΡΑΦΗΜΑ 1: RUNTIME (Για τα Δέντρα) ---
print("Δημιουργία: Κατανομή Διάρκειας...")
plt.figure()
sns.histplot(df['runtime'].dropna(), bins=50, color='skyblue', kde=True)
plt.title('Κατανομή Διάρκειας Ταινιών (Runtime)')
plt.xlabel('Διάρκεια (λεπτά)')
plt.xlim(0, 200) # Εστιάζουμε στο κύριο μέρος
plt.savefig('grafima1_runtime.png', bbox_inches='tight')
plt.close()

# --- ΓΡΑΦΗΜΑ 2: BUDGET vs POPULARITY (Για τα Δέντρα) ---
print("Δημιουργία: Budget vs Popularity...")
plt.figure()
sns.scatterplot(data=df, x='budget', y='popularity', alpha=0.5, color='purple')
plt.title('Συσχέτιση Budget με Δημοτικότητα')
plt.xlabel('Budget ($)')
plt.ylabel('Popularity Score')
plt.xlim(0, 50000000) # Zoom για να μη χαθεί η πληροφορία
plt.ylim(0, 100)
plt.savefig('grafima2_budget_pop.png', bbox_inches='tight')
plt.close()

# --- ΓΡΑΦΗΜΑ 3: RELEASE YEAR (Γενικό) ---
print("Δημιουργία: Ταινίες ανά Έτος...")
if 'release_date' in df.columns:
    df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year
    plt.figure()
    sns.histplot(df['release_year'].dropna(), bins=40, color='orange')
    plt.title('Αριθμός Ταινιών ανά Έτος')
    plt.xlabel('Έτος')
    plt.savefig('grafima3_years.png', bbox_inches='tight')
    plt.close()

# --- ΓΡΑΦΗΜΑ 4: GENRES (Για το LSH) ---
print("Δημιουργία: Top 10 Genres...")
if 'genre_names' in df.columns:
    genres_cleaned = df['genre_names'].astype(str).str.replace(r"[\[\]']", "", regex=True)
    genres_exploded = genres_cleaned.str.split(',').explode().str.strip()
    genres_exploded = genres_exploded[genres_exploded != ''] # Αφαίρεση κενών
    
    top_genres = genres_exploded.value_counts().head(10)

    plt.figure()
    sns.barplot(x=top_genres.values, y=top_genres.index, palette='viridis')
    plt.title('Top 10 Είδη Ταινιών (Genres)')
    plt.xlabel('Πλήθος')
    plt.savefig('grafima4_genres.png', bbox_inches='tight')
    plt.close()

# --- ΓΡΑΦΗΜΑ 5: COMPANIES (Για το LSH) ---
print("Δημιουργία: Top 10 Companies...")
if 'production_company_names' in df.columns:
    comp_cleaned = df['production_company_names'].astype(str).str.replace(r"[\[\]']", "", regex=True)
    comp_exploded = comp_cleaned.str.split(',').explode().str.strip()
    comp_exploded = comp_exploded[comp_exploded != '']
    
    top_comp = comp_exploded.value_counts().head(10)

    plt.figure()
    sns.barplot(x=top_comp.values, y=top_comp.index, palette='magma')
    plt.title('Top 10 Εταιρείες Παραγωγής')
    plt.xlabel('Πλήθος')
    plt.savefig('grafima5_companies.png', bbox_inches='tight')
    plt.close()

print(f"\n✅ ΕΤΟΙΜΑ! Οι 5 εικόνες αποθηκεύτηκαν στο: {save_dir}")